# Import packages and connect to Elasticsearch

In [10]:
from elasticsearch import Elasticsearch
from dotenv import load_dotenv
import os

load_dotenv()
 
openai_api_key=os.getenv('OPENAI_API_KEY')
elastic_user=os.getenv('ES_USER')
elastic_password=os.getenv('ES_PASSWORD')
elastic_endpoint=os.getenv("ES_ENDPOINT")
 
url = f"https://{elastic_user}:{elastic_password}@{elastic_endpoint}:9200"
client = Elasticsearch(url, ca_certs = "./http_ca.crt", verify_certs = True)
 
print(client.info())

{'name': 'liuxgm.local', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'h2QwONxsT4Kt-lTRKmPrhg', 'version': {'number': '8.12.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '1665f706fd9354802c02146c1e6b5c0fbcddfbc9', 'build_date': '2024-01-11T10:05:27.953830042Z', 'build_snapshot': False, 'lucene_version': '9.9.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


# Index some test data

In [11]:
import json

# Load data into a JSON object
with open('query-rules-data.json') as f:
   docs = json.load(f)

operations = []
for doc in docs:
    operations.append({"index": {"_index": "products_index", "_id": doc["id"]}})
    operations.append(doc["content"])
client.bulk(index="products_index", operations=operations, refresh=True)

ObjectApiResponse({'errors': False, 'took': 87, 'items': [{'index': {'_index': 'products_index', '_id': 'us1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'products_index', '_id': 'uk1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'products_index', '_id': 'eu1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'products_index', '_id': 'preview1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}]})

# Search the test data

In [12]:
def pretty_response(response):
    if len(response['hits']['hits']) == 0:
        print('Your search returned no results.')
    else:
        for hit in response['hits']['hits']:
            id = hit['_id']
            score = hit['_score']
            name = hit['_source']['name']
            description = hit['_source']['description']
            price = hit["_source"]["price"]
            currency = hit["_source"]["currency"]
            plug_type = hit["_source"]["plug_type"]
            voltage = hit["_source"]["voltage"]
            pretty_output = (f"\nID: {id}\nName: {name}\nDescription: {description}\nPrice: {price}\nCurrency: {currency}\nPlug type: {plug_type}\nVoltage: {voltage}\nScore: {score}")
            print(pretty_output)

def pretty_ruleset(response):
    print("Ruleset ID: " + response['ruleset_id'])
    for rule in response['rules']:
        rule_id = rule['rule_id']
        type = rule['type']
        print(f"\nRule ID: {rule_id}\n\tType: {type}\n\tCriteria:")
        criteria = rule['criteria']
        for rule_criteria in criteria:
            criteria_type = rule_criteria['type']
            metadata = rule_criteria['metadata']
            values = rule_criteria['values']
            print(f"\t\t{metadata} {criteria_type} {values}")
        ids = rule['actions']['ids']
        print(f"\tPinned ids: {ids}")

## Normal search without using the query rules

In [13]:
response = client.search(index="products_index", query={
    "multi_match": {
        "query": "reliable wireless charger for iPhone",
        "fields": [ "name^5", "description" ]
    }
})

pretty_response(response)


ID: eu1
Name: PureJuice Pro - Wireless Charger suitable for European plugs
Description: PureJuice Pro: Elevating wireless charging. Combining unparalleled charging speeds with elegant design, it promises both rapid and dependable energy for your devices. Embrace the future of wireless charging.
Price: 18.0
Currency: EUR
Plug type: C
Voltage: 230V
Score: 14.5004

ID: preview1
Name: PureJuice Pro - Pre-order next version
Description: Newest version of the PureJuice Pro wireless charger, coming soon! The newest model of the PureJuice Pro boasts a 2x faster charge than the current model, and a sturdier cable with an eighteen month full warranty. We also have a battery backup to charge on-the-go, up to two full charges. Pre-order yours today!
Price: 36.0
Currency: USD
Plug type: ['B', 'C', 'G']
Voltage: ['230V', '120V']
Score: 1.0668641

ID: us1
Name: PureJuice Pro
Description: PureJuice Pro: Experience the pinnacle of wireless charging. Blending rapid charging tech with sleek design, it e

## Creating rules

In [14]:
client.query_ruleset.put(ruleset_id="promotion-rules", rules=[
    {
      "rule_id": "us-charger",
      "type": "pinned",
      "criteria": [
        {
          "type": "contains",
          "metadata": "my_query",
          "values": ["wireless charger"]
        },
        {
          "type": "exact",
          "metadata": "country",
          "values": ["us"]
        }
      ],
      "actions": {
        "ids": [
          "us1"
        ]
      }
    },
    {
      "rule_id": "uk-charger",
      "type": "pinned",
      "criteria": [
        {
          "type": "contains",
          "metadata": "my_query",
          "values": ["wireless charger"]
        },
        {
          "type": "exact",
          "metadata": "country",
          "values": ["uk"]
        }
      ],
      "actions": {
        "ids": [
          "uk1"
        ]
      }
    }
  ])

ObjectApiResponse({'result': 'created'})

## Search with query rules

In [15]:
response = client.query_ruleset.get(ruleset_id="promotion-rules")
pretty_ruleset(response)

Ruleset ID: promotion-rules

Rule ID: us-charger
	Type: pinned
	Criteria:
		my_query contains ['wireless charger']
		country exact ['us']
	Pinned ids: ['us1']

Rule ID: uk-charger
	Type: pinned
	Criteria:
		my_query contains ['wireless charger']
		country exact ['uk']
	Pinned ids: ['uk1']


In [16]:
response = client.search(index="products_index", query={
      "rule_query": {
          "organic": {
              "multi_match": {
                  "query": "reliable wireless charger for iPhone",
                  "fields": [ "name^5", "description" ]
              }
          },
          "match_criteria": {
            "my_query": "reliable wireless charger for iPhone",
            "country": "us"
          },
          "ruleset_id": "promotion-rules"
      }
})

pretty_response(response)


ID: us1
Name: PureJuice Pro
Description: PureJuice Pro: Experience the pinnacle of wireless charging. Blending rapid charging tech with sleek design, it ensures your devices are powered swiftly and safely. The future of charging is here.
Price: 15.0
Currency: USD
Plug type: B
Voltage: 120v
Score: 1.7014122e+38

ID: eu1
Name: PureJuice Pro - Wireless Charger suitable for European plugs
Description: PureJuice Pro: Elevating wireless charging. Combining unparalleled charging speeds with elegant design, it promises both rapid and dependable energy for your devices. Embrace the future of wireless charging.
Price: 18.0
Currency: EUR
Plug type: C
Voltage: 230V
Score: 14.5004

ID: preview1
Name: PureJuice Pro - Pre-order next version
Description: Newest version of the PureJuice Pro wireless charger, coming soon! The newest model of the PureJuice Pro boasts a 2x faster charge than the current model, and a sturdier cable with an eighteen month full warranty. We also have a battery backup to cha